In [3]:
import pandas as pd
import re
import os

# Load the CSV file
csv_path = "../data/SPACE_OncosysOva_ClinicalData_Batch_all_PFS_label.csv" 
data = pd.read_csv(csv_path)
data['study_id'] = data['study_id'].astype(str)
directory_path = "/scratch/project_2003009/space_he/adjacent_slide"  

results = []

for filename in os.listdir(directory_path):
    if filename.endswith(".mrxs"):
        match = re.search(r'S\d+', filename)
        if match:
            study_id = match.group()  # Extracts 'S289'
            matched_row = data[data['study_id'] == study_id]
            
            if not matched_row.empty:
                matched_row = matched_row.assign(filename=filename.replace(".mrxs",""))
                results.append(matched_row)

if results:
    final_result = pd.concat(results, ignore_index=True)
    final_result = final_result[['filename', 'study_id','running_nr','progression','PFS_label']]  # Adjust columns as needed
    final_result = final_result.rename(columns={"filename": "slide", "PFS_label": "er_status_by_ihc", "running_nr":"patient"})
    final_result = final_result[final_result['er_status_by_ihc']!='remove']
    # Exclude slides that case file reading errors
    slides_to_avoid = ['12_S061_pOme_4_HE', '12_S027_pOme_4_HE', '12_S021_pOme_5_HE', '12_S047_pOme_4_HE']
    final_result = final_result[~final_result['slide'].isin(slides_to_avoid)]
    final_result = final_result[final_result['er_status_by_ihc']!='remove']
    # Save the result to a new CSV
    final_result.to_csv("../data/matched_data_space.csv", index=False)
    print("Data matched and saved to 'matched_data.csv'")
else:
    print("No matching files found.")


Data matched and saved to 'matched_data.csv'


In [4]:
!pwd

/projappl/project_2003009/matias/pixel_ai/notebooks


In [5]:
matched_row

,Unnamed: 0,study_id,running_nr,progression,PFS_label


In [4]:
final_result

,slide,study_id,patient,progression,er_status_by_ihc
1,12_S148_pOme_4_HE,S148,50,no,long
3,12_S142_pOvaL_4_HE,S142,47,no,long
4,12_S134_pOvaR_4_HE,S134,41,yes,long
5,12_S085_pOme_4_HE,S085,30,yes,long
7,12_S287_pMes_4_HE,S287,71,no,long
...,...,...,...,...,...
77,12_S145_pOme_BM_4_HE,S145,48,no,long
78,12_S033_pOme_4_HE,S033,17,yes,short
79,12_S008_pOme1_4_HE,S008,5,yes,long
81,12_S109_pOme_5_HE,S109,37,yes,short
